# 3webアプリ作成

目次：https://docs.google.com/document/d/1eDw9PK5Ft0vImu81e7oERB_q-dZvxbFe8bGetZBgEEg/edit

## 3.3 応用実装

### 3.3.1 応用アプリ実装の概要：５つのアプリを作る

### 3.3.2 応用アプリ実装

#### ②株価可視化アプリ

>アプリの概要

<img src="pic/img040.png">  

デプロイ先：https://kabu-step2-ver6-6ijpoom5ze3.streamlit.app/

>動作を構成してる部品の紹介（APIやライブラリ）
API
- yahoo finance
    - 目的：株価情報をyf.Ticker("銘柄番号")の１行で取得できる
    - 参考：https://pypi.org/project/yfinance/

ライブラリ
- streamlit
    - 目的：株価情報を表示するためのフロントエンドを生成する
- pandas
    - 目的：株価APIで得られたデータを扱いやすくするために使う
- yfinance
    - 目的：yahoo financeの株価情報取得手段をライブラリとして提供してくれている。
- altair
    - 目的：pandsで構造化データになった株価情報のデータ可視化のために使う



>ライブラリのインストール

ライブラリ
- streamlit
    - pip install streamlit
- pandas
    - pip install pandas
- yfinance
    - pip install yfinance
- altair
    - pip install altair

>ライブラリの公式ドキュメントの見方や使い方を説明


ライブラリ

- streamlit
    - インストール方法はget startページで確認
    - 参考：https://docs.streamlit.io/library/get-started/installation
    - st.titleなどはコンポーネントページから確認出来る
    - 参考：https://streamlit.io/components
- pandas
    - pd.DataFrame()で使います。
    - 参考：https://pandas.pydata.org/docs/
- yfinance
    - yf.Ticker("銘柄番号")でその銘柄の株価を取得できます。
    - 東証などのシンボルはhttps://support.yahoo-net.jp/PccFinance/s/article/H000006603で検索できる
    - 参考：https://pypi.org/project/yfinance/
- altair
    - alt.Chartでチャート生成
    - オプションは.encode()の中に設定していきます。
    - 参考：https://pypi.org/project/altair/

>それぞれの機能をjupyterで動く最小限を作る

以下がjuptyterにて動く最低限のアプリの流れです。

1. 作るアプリの全体のコードを把握
1. それぞれ機能を解説
    1. 株価取得
    1. チャートによる可視化

1.全体のコード

使うアプリの全体像は下記のようになります。  
これはweb上では動きませんが、手元では動作するアプリです。

以下のコードの動作の流れ
1. 拡張機能であるライブラリをインポート
1. 株価データを要求するための準備
    1. 株価を取得する会社すべてをtickersに代入
    1. 要求する日付区間をdaysに代入
1. 株価データ取得
    1. tickersを元にyf.Ticker("ここ銘柄")で１つ１つデータ取得
    1. dfにデータを格納
    1. dfにtickersにあるすべての企業の株価が格納されている
1. 株価データ整理
    1. 欲しい銘柄だけをcompaniesに代入
    1. companiesを使って取得した情報の中から情報を抽出
1. チャートで可視化
    1. チャートの表示範囲の値をyminとymaxに代入
    1. alt.Chartを使って抽出した株価データを可視化

In [8]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
import pandas as pd # データフレームを扱う機能をインポート
import yfinance as yf # yahoo financeから株価情報を取得するための機能をインポート
import altair as alt # チャート可視化機能をインポート

#取得する銘柄の名前とキーを変換する一覧を設定
#東証などのシンボルはhttps://support.yahoo-net.jp/PccFinance/s/article/H000006603で検索できる
tickers = {
    'apple': 'AAPL',
    'facebook': 'META',
    'google': 'GOOGL',
    'microsoft': 'MSFT',
    'netflix': 'NFLX',
    'amazon': 'AMZN',
    'TOTO': '5332.T',
    'TOYOTA': '7203.T',
}

print('株価可視化アプリ')
print("こちらは株価可視化ツールです。以下のオプションから表示日数を指定できます。")# サイドバーに表示
print("表示日数選択") # サイドバーに表示

days = 10 # 取得するための日数daysに代入

print(f"過去 {days}日間 の株価") # 取得する日数を表示


def get_data(days, tickers):
    df = pd.DataFrame() # 株価を代入するための空箱を用意

    # 選択した株価の数だけ yf.Tickerでリクエストしてデータを取得する
    for company in tickers.keys():
        tkr = yf.Ticker(tickers[company]) # 設定した銘柄一覧でリクエストの為の7203.Tなどに変換をして、それをyf.Tickerで株価リクエスト
        hist = tkr.history(period=f'{days}d') # スライドバーで指定した日数で取得した情報を絞る
        hist.index = hist.index.strftime('%d %B %Y') # indexを日付のフォーマットに変更
        hist = hist[['Close']] # データを終値だけ抽出
        hist.columns = [company] # データのカラムをyf.Tickerのリクエストした企業名に設定
        hist = hist.T # 欲しい情報が逆なので、転置する
        hist.index.name = 'Name' # indexの名前をNameにする
        df = pd.concat([df, hist]) # 用意した空のデータフレームに設定したhistのデータを結合する
    return df # 返り値としてdfを返す

df = get_data(days, tickers) # リクエストする企業一覧すべてと変換するtickersを引数に株価取得





# 企業名の配列を生成し、companiesに代入
companies = ['google', 'apple','TOYOTA']

data = df.loc[companies] # 取得したデータから抽出するための配列で絞ってdataに代入
print("株価 ", data.sort_index()) # dataにあるindexを表示
data = data.T.reset_index() # dataを抽出して転置

# 企業ごとの別々のカラムにデータを表示する必要ないので企業を１つのカラムに統一
data = pd.melt(data, id_vars=['Date']).rename(
    columns={'value': 'Stock Prices'}
)


# チャートに表示する範囲、それぞれをymin, ymaxに代入
print("株価の範囲指定") # サイドバーに表示
ymin = 0
ymax = 3000

# dataとスライドバーで設定した最大最小値を元にalt.Chartを使って株価チャートを作成
chart = (
    alt.Chart(data)
    .mark_line(opacity=0.8, clip=True)
    .encode(
        x="Date:T",
        y=alt.Y("Stock Prices:Q", stack=None, scale=alt.Scale(domain=[ymin, ymax])),
        color='Name:N'
    )
)

# 作成したチャートをstreamlitで表示
display(chart)

株価可視化アプリ
こちらは株価可視化ツールです。以下のオプションから表示日数を指定できます。
表示日数選択
過去 10日間 の株価
株価  Date    27 June 2023  28 June 2023  29 June 2023  30 June 2023  03 July 2023  \
Name                                                                           
TOYOTA           NaN           NaN   2295.500000   2308.500000   2325.500000   
apple     188.059998        189.25    189.589996    193.970001    192.460007   
google    118.330002        120.18    119.099998    119.699997    119.900002   

Date    05 July 2023  06 July 2023  07 July 2023  10 July 2023  11 July 2023  \
Name                                                                           
TOYOTA   2341.000000   2327.000000   2299.500000   2260.000000   2211.000000   
apple     191.330002    191.809998    190.679993    188.610001    188.080002   
google    121.750000    120.110001    119.480003    116.449997    117.139999   

Date    04 July 2023  12 July 2023  
Name                                
TOYOTA        2338.0        2195.0  
apple            

alt.Chart(...)

2.それぞれ機能を解説

2.1株価取得

具体例として、TOYOTAを参考に株価を取得します。  
必要な情報は銘柄：7203.T だけです。

yf.Ticker(ここ銘柄)で情報を取得できます。  
可視化はtkr.history(period='ここ欲しい区間') で構造化データを取得できます。

In [9]:
thr = yf.Ticker("7203.T") # 設定した銘柄一覧でリクエストの為の7203.Tなどに変換をして、それをyf.Tickerで株価リクエスト
hist = thr.history(period= "10d")
display(hist)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-06-29 00:00:00+09:00,2302.0,2311.0,2278.5,2295.5,27070800,0.0,0.0
2023-06-30 00:00:00+09:00,2320.0,2323.5,2285.5,2308.5,37428700,0.0,0.0
2023-07-03 00:00:00+09:00,2308.0,2329.5,2303.5,2325.5,25421400,0.0,0.0
2023-07-04 00:00:00+09:00,2332.5,2344.5,2324.5,2338.0,26554700,0.0,0.0
2023-07-05 00:00:00+09:00,2336.0,2341.5,2322.5,2341.0,20643100,0.0,0.0
2023-07-06 00:00:00+09:00,2350.0,2358.0,2317.0,2327.0,28608700,0.0,0.0
2023-07-07 00:00:00+09:00,2323.5,2324.0,2292.0,2299.5,26236300,0.0,0.0
2023-07-10 00:00:00+09:00,2270.0,2290.0,2255.0,2260.0,28824200,0.0,0.0
2023-07-11 00:00:00+09:00,2256.0,2258.5,2204.0,2211.0,29078400,0.0,0.0


数字だけでは、どの企業のデータかわからないので、カラムにTOYOTAの企業名を入れ、終値だけ取得します。

In [16]:
# indexを日付のフォーマットに変更
# データを終値だけ抽出
# データのカラムをyf.Tickerのリクエストした企業名に設定
hist.index = hist.index.strftime('%d %B %Y')
hist = hist[['Close']]
hist.columns = ["TOYOTA"]
display(hist)


,TOYOTA
Date,
29 June 2023,2295.5
30 June 2023,2308.5
03 July 2023,2325.5
04 July 2023,2338.0
05 July 2023,2341.0
06 July 2023,2327.0
07 July 2023,2299.5
10 July 2023,2260.0
11 July 2023,2211.0


注意点として、複数の株価データが日数が違うデータもindexであるDateに順番に格納されてしまうので、  
対策として日付をカラムにします。

.Tで転置ができます。

In [17]:
# 欲しい情報が逆なので、転置する
 # indexの名前をNameにする

hist = hist.T
hist.index.name = 'Name'
display(hist)

Date,29 June 2023,30 June 2023,03 July 2023,04 July 2023,05 July 2023,06 July 2023,07 July 2023,10 July 2023,11 July 2023,12 July 2023
Name,,,,,,,,,,
TOYOTA,2295.5,2308.5,2325.5,2338.0,2341.0,2327.0,2299.5,2260.0,2211.0,2195.5


これで株価データの取得が完了しました。

2.2チャートによる可視化

チャートにするために、構造化データを日付をx軸、株価をy軸に変更します。
複数のデータがあった方がわかりやすいので、チャートのための構造化データを以下のコードで作成します。

In [20]:
# tickersに設定した銘柄一覧でリクエストする一覧を設定
 # 株価を代入するための空箱を用意

 # 設定した銘柄一覧でリクエストの為の7203.Tなどに変換をして、それをyf.Tickerで株価リクエスト
    # スライドバーで指定した日数で取得した情報を絞る
    # indexを日付のフォーマットに変更
    # データを終値だけ抽出
    # データのカラムをyf.Tickerのリクエストした企業名に設定
    # 欲しい情報が逆なので、転置する
     # indexの名前をNameにする
    # 用意した空のデータフレームに設定したhistのデータを結合する

tickers = {
    'apple': 'AAPL',
    'facebook': 'META',
    'google': 'GOOGL',
    'microsoft': 'MSFT',
    'netflix': 'NFLX',
    'amazon': 'AMZN',
    'TOTO': '5332.T',
    'TOYOTA': '7203.T',
}
days = 10

df = pd.DataFrame()

for company in tickers.keys():
    thr = yf.Ticker(tickers[company])
    hist = thr.history(period=f'{days}d')
    hist.index = hist.index.strftime('%d %B %Y')
    hist = hist[['Close']]
    hist.columns = [company]
    hist = hist.T
    hist.index.name ='Name'
    df = pd.concat([df, hist])

display(df)

Date,27 June 2023,28 June 2023,29 June 2023,30 June 2023,03 July 2023,05 July 2023,06 July 2023,07 July 2023,10 July 2023,11 July 2023,04 July 2023,12 July 2023
Name,,,,,,,,,,,,
apple,188.059998,189.250000,189.589996,193.970001,192.460007,191.330002,191.809998,190.679993,188.610001,188.080002,NaN,NaN
facebook,287.049988,285.290009,281.529999,286.980011,286.019989,294.369995,291.989990,290.529999,294.100006,298.290009,NaN,NaN
google,118.330002,120.180000,119.099998,119.699997,119.900002,121.750000,120.110001,119.480003,116.449997,117.139999,NaN,NaN
microsoft,334.570007,335.850006,335.049988,340.540009,337.989990,338.149994,341.269989,337.220001,331.829987,332.470001,NaN,NaN
netflix,417.079987,429.839996,428.239990,440.489990,441.440002,445.899994,438.839996,438.100006,441.709991,440.209991,NaN,NaN
amazon,129.179993,129.039993,127.900002,130.360001,130.220001,130.380005,128.360001,129.779999,127.129997,128.779999,NaN,NaN
TOTO,NaN,NaN,4329.000000,4325.000000,4369.000000,4303.000000,4240.000000,4197.000000,4198.000000,4217.000000,4332.0,4221.0
TOYOTA,NaN,NaN,2295.500000,2308.500000,2325.500000,2341.000000,2327.000000,2299.500000,2260.000000,2211.000000,2338.0,2198.5


構造化データを日付をx軸、株価をy軸するために転置します。

In [22]:
# 全体像のコードと一致させるために変数名を変更します。
 # dataを抽出して転置

data = df
data = data.T.reset_index()
display(data)


Name,Date,apple,facebook,google,microsoft,netflix,amazon,TOTO,TOYOTA
0,27 June 2023,188.059998,287.049988,118.330002,334.570007,417.079987,129.179993,NaN,NaN
1,28 June 2023,189.250000,285.290009,120.180000,335.850006,429.839996,129.039993,NaN,NaN
2,29 June 2023,189.589996,281.529999,119.099998,335.049988,428.239990,127.900002,4329.0,2295.5
3,30 June 2023,193.970001,286.980011,119.699997,340.540009,440.489990,130.360001,4325.0,2308.5
4,03 July 2023,192.460007,286.019989,119.900002,337.989990,441.440002,130.220001,4369.0,2325.5
5,05 July 2023,191.330002,294.369995,121.750000,338.149994,445.899994,130.380005,4303.0,2341.0
6,06 July 2023,191.809998,291.989990,120.110001,341.269989,438.839996,128.360001,4240.0,2327.0
7,07 July 2023,190.679993,290.529999,119.480003,337.220001,438.100006,129.779999,4197.0,2299.5
8,10 July 2023,188.610001,294.100006,116.449997,331.829987,441.709991,127.129997,4198.0,2260.0
9,11 July 2023,188.080002,298.290009,117.139999,332.470001,440.209991,128.779999,4217.0,2211.0


バラバラのデータである必要がないので、企業を１つにまとめます。  
使い方は以下のようになります。

pd.melt(ここにデータ, id_vars=[変更した良くない列]).rename(
    columns={'value': '新しいカラムの名前'}
)

In [23]:
# 企業ごとの別々のカラムにデータを表示する必要ないので企業を１つのカラムに統一
data = pd.melt(data, id_vars=['Date']).rename(
    columns={'value': 'Stock Prices'}
)
display(data)


,Date,Name,Stock Prices
0,27 June 2023,apple,188.059998
1,28 June 2023,apple,189.250000
2,29 June 2023,apple,189.589996
3,30 June 2023,apple,193.970001
4,03 July 2023,apple,192.460007
...,...,...,...
91,07 July 2023,TOYOTA,2299.500000
92,10 July 2023,TOYOTA,2260.000000
93,11 July 2023,TOYOTA,2211.000000
94,04 July 2023,TOYOTA,2338.000000


2.2チャートによる可視化

alt.Chartを使って、チャートデータを作成します。  
指定している設定は以下のようになります。

- チャートに使うデータをdata
- xのラベルにDate　色種類は:T
- yのラベルにStock Prices　色種類は:Q　範囲0, 3000
- 全体の色種類指定:N

In [24]:
# chartで表示するデータを入れる

# 作成したチャートをstreamlitで表示

chart = (
    alt.Chart(data).mark_line(opacity=0.8, clip=True).encode(
        x="Date:T",
        y=alt.Y("Stock Prices:Q", stack=None, scale= alt.Scale(domain = [0, 3000])),
        color='Name:N'
    )
)

display(chart)

alt.Chart(...)

以上がアプリの動作部分です。

>streamlitに実装する場合の使う機能

今はユーザーがブラウザ上で選択したりなど、想定していませんが、  
選択肢の表示などのコンポーネントを紹介します。

※streamlitはjupyterでは動作せず、実行ファイルでのみ動作します

タイトルは  
st.title("ここにタイトル")

ただの表示は  
st.write("ここに表示したい文字")

ただし、再読み込みしない限り、この表示された文字は変更されません。

選択肢は  st.selectbox("ここに表示したい項目名",ここに配列のデータ)  
でできます。

サイドバーに配置したい場合は、   
st.slider.write("ここに表示したい文字")のように.sliderをstのうしろにつけます。

スライドバーの表示は  
st.sidebar.slider('ここに表示したい文字', 最小値の数字, 最大値の数字, 初期の位置の値)  
でできます。

チャートの表示は  
st.altair_chart(ここにちゃーとのデータ, use_container_width=True)でできます。

以上の機能を使ってwebアプリにしていきます。

>（演習）streamlitに実装しましょう！

手元で動いたアプリをstreamlitで動くようにします。  
この演習では必ず実行ファイルにコーディングしてください。

※streamlit run ファイル名で実行する時は必ず実行ファイルで実行してください。

実装の流れ
1. 取得した全株価から抽出する条件companiesの内容をセレクターで取得
    1. 構造化データから選択肢抽出
    1. companiesにセレクトボックスの値が入るようにする
1. 固定数字をスライドバーで変数へ
1. ターミナルに出力していた文字などをブラウザに出力

以下の手元で動いたコードをstreamlitで動くようにします。

In [26]:
# フロントエンドを扱うstreamlitの機能をインポート
# データフレームを扱う機能をインポート
# yahoo financeから株価情報を取得するための機能をインポート
# チャート可視化機能をインポート

import streamlit as st
import pandas as pd
import yfinance as yf
import altair as alt

#取得する銘柄の名前とキーを変換する一覧を設定
#東証などのシンボルはhttps://support.yahoo-net.jp/PccFinance/s/article/H000006603で検索できる

tickers = {
      'apple': 'AAPL',
      'facebook': 'META',
      'google': 'GOOGL',
      'microsoft': 'MSFT',
      'netflix': 'NFLX',
      'amazon': 'AMZN',
      'TOTO': '5332.T',
      'TOYOTA': '7203.T',
}

print('株価可視化アプリ')
print("こちらは株価可視化ツールです。以下のオプションから表示日数を指定できます。")# サイドバーに表示
print("表示日数選択") # サイドバーに表示


# 取得するための日数daysに代入

 # 取得する日数を表示

days = 10

print(f"過去 {days}日間 の株価") 

 # 株価を代入するための空箱を用意

 

def get_data(days, tickers):
    df = pd.DataFrame() # 株価を代入するための空箱を用意

    # 選択した株価の数だけ yf.Tickerでリクエストしてデータを取得する
    for company in tickers.keys():
        tkr = yf.Ticker(tickers[company]) # 設定した銘柄一覧でリクエストの為の7203.Tなどに変換をして、それをyf.Tickerで株価リクエスト
        hist = tkr.history(period=f'{days}d') # スライドバーで指定した日数で取得した情報を絞る
        hist.index = hist.index.strftime('%d %B %Y') # indexを日付のフォーマットに変更
        hist = hist[['Close']] # データを終値だけ抽出
        hist.columns = [company] # データのカラムをyf.Tickerのリクエストした企業名に設定
        hist = hist.T # 欲しい情報が逆なので、転置する
        hist.index.name = 'Name' # indexの名前をNameにする
        df = pd.concat([df, hist]) # 用意した空のデータフレームに設定したhistのデータを結合する
    return df # 返り値としてdfを返す

df = get_data(days, tickers) # リクエストする企業一覧すべてと変換するtickersを引数に株価取得





# 企業名の配列を生成し、companiesに代入
companies = ['google', 'apple','TOYOTA']


data =df.loc[companies] # 取得したデータから抽出するための配列で絞ってdataに代入
 # dataにあるindexを表示
 # dataを抽出して転置

print("株価" , data.sort_index()) # dataを抽出して転置
data = data.T.reset_index() # dataを抽出して転置

# 企業ごとの別々のカラムにデータを表示する必要ないので企業を１つのカラムに統一
data = pd.melt(data, id_vars=['Date']).rename(
    columns={'value': 'Stock Prices'}
)

# チャートに表示する範囲、それぞれをymin, ymaxに代入
# サイドバーに表示
ymin = 0
ymax = 3000

# dataとスライドバーで設定した最大最小値を元にalt.Chartを使って株価チャートを作成
chart = (
    alt.Chart(data)
    .mark_line(opacity=0.8, clip=True)
    .encode(
        x="Date:T",
        y=alt.Y("Stock Prices:Q", stack=None, scale=alt.Scale(domain=[ymin, ymax])),
        color='Name:N'
    )
)

# 作成したチャートをstreamlitで表示

display(chart)

株価可視化アプリ
こちらは株価可視化ツールです。以下のオプションから表示日数を指定できます。
表示日数選択
過去 10日間 の株価
株価 Date    27 June 2023  28 June 2023  29 June 2023  30 June 2023  03 July 2023  \
Name                                                                           
TOYOTA           NaN           NaN   2295.500000   2308.500000   2325.500000   
apple     188.059998        189.25    189.589996    193.970001    192.460007   
google    118.330002        120.18    119.099998    119.699997    119.900002   

Date    05 July 2023  06 July 2023  07 July 2023  10 July 2023  11 July 2023  \
Name                                                                           
TOYOTA   2341.000000   2327.000000   2299.500000   2260.000000   2211.000000   
apple     191.330002    191.809998    190.679993    188.610001    188.080002   
google    121.750000    120.110001    119.480003    116.449997    117.139999   

Date    04 July 2023  12 July 2023  
Name                                
TOYOTA        2338.0        2200.5  
apple            N

alt.Chart(...)

1.取得した全株価から抽出する条件companiesの内容をセレクターで取得

df.indexで構造化データから取得した企業のリストを取得できるので、これをlist関数を使って配列にする。

In [27]:
list(df.index)

['apple',
 'facebook',
 'google',
 'microsoft',
 'netflix',
 'amazon',
 'TOTO',
 'TOYOTA']

セレクトボックスの値をcompaniesに代入する。

使い方は  
st.selectbox("ここに表示したい項目名",ここに配列のデータ,初期で選択されている配列)

初期で選択されている配列はどれでも構いませんが、ここでは['google', 'apple','TOYOTA']を指定しておきます。

In [ ]:
companies = st.multiselect(
    '会社名を選択してください。',
    list(df.index),
    ['google', 'apple','TOYOTA'],
)

2.固定数字をスライドバーで変数へ

10日で固定しているdaysとチャートの表示範囲ymin,ymaxをスライドバーから取得します。  
変更する固定数字は以下の３つです。

In [ ]:
days = 10

ymin = 0
ymax = 3000

スライドバーの使い方は以下のようになります。  
st.slider('ここに表示したい文字', 最小値の数字, 最大値の数字, 初期の位置の値) 

また１つのスライドバーから２つの値を取得する場合は、

小さい方の数字, 大きいの方の数字 = st.sidebar.slider(
    'ここに表示したい文字',
    最小値の数字, 最大値の数字, (最小値の数字, 最大値の数字)
)

でできます。

In [ ]:
days = st.sidebar.slider('日数', 1, 50, 20)

ymin, ymax = st.slider(
    '範囲を指定してください。',
    0.0, 5000.0, (0.0, 5000.0)
)

streamlitに実装したアプリをまとめると、以下のようになります。

In [ ]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
import pandas as pd # データフレームを扱う機能をインポート
import yfinance as yf # yahoo financeから株価情報を取得するための機能をインポート
import altair as alt # チャート可視化機能をインポート

#取得する銘柄の名前とキーを変換する一覧を設定
#東証などのシンボルはhttps://support.yahoo-net.jp/PccFinance/s/article/H000006603で検索できる
tickers = {
    'apple': 'AAPL',
    'facebook': 'META',
    'google': 'GOOGL',
    'microsoft': 'MSFT',
    'netflix': 'NFLX',
    'amazon': 'AMZN',
    'TOTO': '5332.T',
    'TOYOTA': '7203.T',
}

print('株価可視化アプリ')
print("こちらは株価可視化ツールです。以下のオプションから表示日数を指定できます。")# サイドバーに表示
print("表示日数選択") # サイドバーに表示

days = st.sidebar.slider('日数', 1, 50, 20) # 取得するための日数daysに代入

print(f"過去 {days}日間 の株価") # 取得する日数を表示


def get_data(days, tickers):
    df = pd.DataFrame() # 株価を代入するための空箱を用意

    # 選択した株価の数だけ yf.Tickerでリクエストしてデータを取得する
    for company in tickers.keys():
        tkr = yf.Ticker(tickers[company]) # 設定した銘柄一覧でリクエストの為の7203.Tなどに変換をして、それをyf.Tickerで株価リクエスト
        hist = tkr.history(period=f'{days}d') # スライドバーで指定した日数で取得した情報を絞る
        hist.index = hist.index.strftime('%d %B %Y') # indexを日付のフォーマットに変更
        hist = hist[['Close']] # データを終値だけ抽出
        hist.columns = [company] # データのカラムをyf.Tickerのリクエストした企業名に設定
        hist = hist.T # 欲しい情報が逆なので、転置する
        hist.index.name = 'Name' # indexの名前をNameにする
        df = pd.concat([df, hist]) # 用意した空のデータフレームに設定したhistのデータを結合する
    return df # 返り値としてdfを返す

df = get_data(days, tickers) # リクエストする企業一覧すべてと変換するtickersを引数に株価取得





# 企業名の配列を生成し、companiesに代入
companies = st.multiselect(
    '会社名を選択してください。',
    list(df.index),
    ['google', 'apple','TOYOTA'],
)

data = df.loc[companies] # 取得したデータから抽出するための配列で絞ってdataに代入
print("株価 ", data.sort_index()) # dataにあるindexを表示
data = data.T.reset_index() # dataを抽出して転置

# 企業ごとの別々のカラムにデータを表示する必要ないので企業を１つのカラムに統一
data = pd.melt(data, id_vars=['Date']).rename(
    columns={'value': 'Stock Prices'}
)


# チャートに表示する範囲、それぞれをymin, ymaxに代入
print("株価の範囲指定") # サイドバーに表示


ymin, ymax = st.slider(
    '範囲を指定してください。',
    0.0, 5000.0, (0.0, 5000.0)
)

# dataとスライドバーで設定した最大最小値を元にalt.Chartを使って株価チャートを作成
chart = (
    alt.Chart(data)
    .mark_line(opacity=0.8, clip=True)
    .encode(
        x="Date:T",
        y=alt.Y("Stock Prices:Q", stack=None, scale=alt.Scale(domain=[ymin, ymax])),
        color='Name:N'
    )
)

# 作成したチャートをstreamlitで表示
display(chart)

3.ターミナルに出力していた文字などをブラウザに出力

最後にprintでターミナルで出力していた  
タイトルや文字やチャートを以下の４つのコードを使ってブラウザに出力します。

- st.title
- st.write
- st.altair_chart

In [ ]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
import pandas as pd # データフレームを扱う機能をインポート
import yfinance as yf # yahoo financeから株価情報を取得するための機能をインポート
import altair as alt # チャート可視化機能をインポート

#取得する銘柄の名前とキーを変換する一覧を設定
#東証などのシンボルはhttps://support.yahoo-net.jp/PccFinance/s/article/H000006603で検索できる
tickers = {
    'apple': 'AAPL',
    'facebook': 'META',
    'google': 'GOOGL',
    'microsoft': 'MSFT',
    'netflix': 'NFLX',
    'amazon': 'AMZN',
    'TOTO': '5332.T',
    'TOYOTA': '7203.T',
}

st.title('株価可視化アプリ') # タイトル

st.write("こちらは株価可視化ツールです。以下のオプションから表示日数を指定できます。") # サイドバーに表示

st.write("表示日数選択") # サイドバーに表示

days = st.slider('日数', 1, 50, 20) # サイドバーに表示　取得するための日数をスライドバーで表示し、daysに代入

st.write(f"過去 {days}日間 の株価") # 取得する日数を表示

# @st.cache_dataで読み込みが早くなるように処理を保持しておける
@st.cache_data
def get_data(days, tickers):
    df = pd.DataFrame() # 株価を代入するための空箱を用意

    # 選択した株価の数だけ yf.Tickerでリクエストしてデータを取得する
    for company in tickers.keys():
        tkr = yf.Ticker(tickers[company]) # 設定した銘柄一覧でリクエストの為の7203.Tなどに変換をして、それをyf.Tickerで株価リクエスト
        hist = tkr.history(period=f'{days}d') # スライドバーで指定した日数で取得した情報を絞る
        hist.index = hist.index.strftime('%d %B %Y') # indexを日付のフォーマットに変更
        hist = hist[['Close']] # データを終値だけ抽出
        hist.columns = [company] # データのカラムをyf.Tickerのリクエストした企業名に設定
        hist = hist.T # 欲しい情報が逆なので、転置する
        hist.index.name = 'Name' # indexの名前をNameにする
        df = pd.concat([df, hist]) # 用意した空のデータフレームに設定したhistのデータを結合する
    return df # 返り値としてdfを返す

# チャートに表示する範囲をスライドで表示し、それぞれをymin, ymaxに代入
st.write("株価の範囲指定") # サイドバーに表示
ymin, ymax = st.slider(
    '範囲を指定してください。',
    0.0, 5000.0, (0.0, 5000.0)
) # サイドバーに表示

df = get_data(days, tickers) # リクエストする企業一覧すべてと変換するtickersを引数に株価取得


# 取得したデータから抽出するための配列を生成し、companiesに代入
companies = st.multiselect(
    '会社名を選択してください。',
    list(df.index),
    ['google', 'apple','TOYOTA'], # 最初に表示する企業名を設定
)


data = df.loc[companies] # 取得したデータから抽出するための配列で絞ってdataに代入
st.write("株価 ", data.sort_index()) # dataにあるindexを表示
data = data.T.reset_index() # dataを抽出して転置

# 企業ごとの別々のカラムにデータを表示する必要ないので企業を１つのカラムに統一
data = pd.melt(data, id_vars=['Date']).rename(
    columns={'value': 'Stock Prices'}
)

# dataとスライドバーで設定した最大最小値を元にalt.Chartを使って株価チャートを作成
chart = (
    alt.Chart(data)
    .mark_line(opacity=0.8, clip=True)
    .encode(
        x="Date:T",
        y=alt.Y("Stock Prices:Q", stack=None, scale=alt.Scale(domain=[ymin, ymax])),
        color='Name:N'
    )
)

# 作成したチャートをstreamlitで表示
st.altair_chart(chart, use_container_width=True)

最後にサイドバーに表示する項目に.sidebarをつけます。

In [ ]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
import pandas as pd # データフレームを扱う機能をインポート
import yfinance as yf # yahoo financeから株価情報を取得するための機能をインポート
import altair as alt # チャート可視化機能をインポート

#取得する銘柄の名前とキーを変換する一覧を設定
#東証などのシンボルはhttps://support.yahoo-net.jp/PccFinance/s/article/H000006603で検索できる
tickers = {
    'apple': 'AAPL',
    'facebook': 'META',
    'google': 'GOOGL',
    'microsoft': 'MSFT',
    'netflix': 'NFLX',
    'amazon': 'AMZN',
    'TOTO': '5332.T',
    'TOYOTA': '7203.T',
}

st.title('株価可視化アプリ') # タイトル

st.sidebar.write("こちらは株価可視化ツールです。以下のオプションから表示日数を指定できます。") # サイドバーに表示

st.sidebar.write("表示日数選択") # サイドバーに表示

days = st.sidebar.slider('日数', 1, 50, 20) # サイドバーに表示　取得するための日数をスライドバーで表示し、daysに代入

st.write(f"過去 {days}日間 の株価") # 取得する日数を表示

# @st.cache_dataで読み込みが早くなるように処理を保持しておける
@st.cache_data
def get_data(days, tickers):
    df = pd.DataFrame() # 株価を代入するための空箱を用意

    # 選択した株価の数だけ yf.Tickerでリクエストしてデータを取得する
    for company in tickers.keys():
        tkr = yf.Ticker(tickers[company]) # 設定した銘柄一覧でリクエストの為の7203.Tなどに変換をして、それをyf.Tickerで株価リクエスト
        hist = tkr.history(period=f'{days}d') # スライドバーで指定した日数で取得した情報を絞る
        hist.index = hist.index.strftime('%d %B %Y') # indexを日付のフォーマットに変更
        hist = hist[['Close']] # データを終値だけ抽出
        hist.columns = [company] # データのカラムをyf.Tickerのリクエストした企業名に設定
        hist = hist.T # 欲しい情報が逆なので、転置する
        hist.index.name = 'Name' # indexの名前をNameにする
        df = pd.concat([df, hist]) # 用意した空のデータフレームに設定したhistのデータを結合する
    return df # 返り値としてdfを返す

# チャートに表示する範囲をスライドで表示し、それぞれをymin, ymaxに代入
st.sidebar.write("株価の範囲指定") # サイドバーに表示
ymin, ymax = st.sidebar.slider(
    '範囲を指定してください。',
    0.0, 5000.0, (0.0, 5000.0)
) # サイドバーに表示

df = get_data(days, tickers) # リクエストする企業一覧すべてと変換するtickersを引数に株価取得


# 取得したデータから抽出するための配列を生成し、companiesに代入
companies = st.multiselect(
    '会社名を選択してください。',
    list(df.index),
    ['google', 'apple','TOYOTA'], # 最初に表示する企業名を設定
)


data = df.loc[companies] # 取得したデータから抽出するための配列で絞ってdataに代入
st.write("株価 ", data.sort_index()) # dataにあるindexを表示
data = data.T.reset_index() # dataを抽出して転置

# 企業ごとの別々のカラムにデータを表示する必要ないので企業を１つのカラムに統一
data = pd.melt(data, id_vars=['Date']).rename(
    columns={'value': 'Stock Prices'}
)

# dataとスライドバーで設定した最大最小値を元にalt.Chartを使って株価チャートを作成
chart = (
    alt.Chart(data)
    .mark_line(opacity=0.8, clip=True)
    .encode(
        x="Date:T",
        y=alt.Y("Stock Prices:Q", stack=None, scale=alt.Scale(domain=[ymin, ymax])),
        color='Name:N'
    )
)

# 作成したチャートをstreamlitで表示
st.altair_chart(chart, use_container_width=True)